# Persisting AMFabric

In [ ]:
from src.amfabric import AMFabric
from src.sdr import SDR


# library to plot data
#
import plotly.graph_objects as go


# plot the fabric structure
#
def plot_fabric_3d(fabric, title, coords_to_highlight=None, neurons_to_plot=None, short_term_memory=1):
    x_node = []
    y_node = []
    z_node = []
    colors = []
    x_edge = []
    y_edge = []
    z_edge = []
    labels = []
    sizes = []
    pairs = set()
    nos_winners = 0
    sum_distance = 0.0
    sum_mapped = 0
    
    # 
    if neurons_to_plot is None:
        neurons_to_plot = [n for n in range(short_term_memory)]
    
    if coords_to_highlight is None:
        coords_to_highlight = list(fabric['neuro_columns'].keys())
        
    for coord_id in fabric['neuro_columns'].keys():

        if fabric['neuro_columns'][coord_id]['n_bmu'] > 0:
            nos_winners += 1
            sum_distance += fabric['neuro_columns'][coord_id]['mean_distance']
            sum_mapped += fabric['neuro_columns'][coord_id]['n_bmu']

        for neuron_id in neurons_to_plot:
            if 'trade:*:{}:{}:{}:{}:{}'.format('has_rgb', 'r', neuron_id, 'rgb', 'r') in fabric['neuro_columns'][coord_id]['neuro_column']:

                # if neuron in the top of the column and has been trained then increase size
                #
                if neuron_id == neurons_to_plot[0] and fabric['neuro_columns'][coord_id]['n_bmu'] + fabric['neuro_columns'][coord_id]['n_nn'] > 0:
                    sizes.append(20 + fabric['neuro_columns'][coord_id]['n_bmu'])
                else:
                    sizes.append(15)

                nc = fabric['neuro_columns'][coord_id]['neuro_column']

                x = fabric['neuro_columns'][coord_id]['coord'][0]
                y = fabric['neuro_columns'][coord_id]['coord'][1]
                z = max(neurons_to_plot) - neuron_id

                x_node.append(x)
                y_node.append(y)
                z_node.append(z)

                if coord_id in coords_to_highlight:
                    r = round(nc['trade:*:{}:{}:{}:{}:{}'.format('has_rgb', 'r', neuron_id, 'rgb', 'r')]['numeric'])
                    g = round(nc['trade:*:{}:{}:{}:{}:{}'.format('has_rgb', 'g', neuron_id, 'rgb', 'g')]['numeric'])
                    b = round(nc['trade:*:{}:{}:{}:{}:{}'.format('has_rgb', 'b', neuron_id, 'rgb', 'b')]['numeric'])
                    opacity = 1.0
                else:
                    r = 255
                    g = 255
                    b = 255
                    opacity = 0.7
                colors.append('rgba({},{},{},{})'.format(r, g, b, opacity))

                labels.append('NeuroColumn: {}<br>Neuron: {}<br>n_BMU: {}<br>last_bmu: {}<br>n_nn: {}<br>last_nn: {}<br>mean_dist: {}<br>mean_sim: {}<br>r: {}<br>g:{}<br>b:{}<br>'.format(coord_id, 
                                                                                                                           neuron_id, 
                                                                                                                           fabric['neuro_columns'][coord_id]['n_bmu'],
                                                                                                                           fabric['neuro_columns'][coord_id]['last_bmu'],
                                                                                                                           fabric['neuro_columns'][coord_id]['n_nn'],
                                                                                                                           fabric['neuro_columns'][coord_id]['last_nn'],
                                                                                                                           fabric['neuro_columns'][coord_id]['mean_distance'] if 'mean_distance' in fabric['neuro_columns'][coord_id] else None,
                                                                                                                           fabric['neuro_columns'][coord_id]['mean_similarity'] if 'mean_similarity' in fabric['neuro_columns'][coord_id] else None,
                                                                                                                           r, g, b))
                
                # connect neurons in same column
                #
                if neuron_id < neurons_to_plot[-1]:
                    pair = (min((x, y, neuron_id), (x, y, neuron_id + 1)), max((x, y, neuron_id), (x, y, neuron_id + 1)))

                    pairs.add(pair)

                    x_edge.append(x)
                    x_edge.append(x)
                    x_edge.append(None)

                    y_edge.append(y)
                    y_edge.append(y)
                    y_edge.append(None)

                    nn_z = max(neurons_to_plot) - neuron_id - 1
                    z_edge.append(z)
                    z_edge.append(nn_z)
                    z_edge.append(None)

                # connect neuro_columns
                #
                for nn_id in fabric['neuro_columns'][coord_id]['nn']:
                    nn_x = fabric['neuro_columns'][nn_id]['coord'][0]
                    nn_y = fabric['neuro_columns'][nn_id]['coord'][1]

                    pair = (min((x, y, neuron_id), (nn_x, nn_y, neuron_id)), max((x, y, neuron_id), (nn_x, nn_y, neuron_id)))

                    if pair not in pairs:
                        pairs.add(pair)
                        nn_z = max(neurons_to_plot) - neuron_id
                        x_edge.append(x)
                        x_edge.append(nn_x)
                        x_edge.append(None)

                        y_edge.append(y)
                        y_edge.append(nn_y)
                        y_edge.append(None)

                        z_edge.append(z)
                        z_edge.append(nn_z)
                        z_edge.append(None)

    neuron_scatter = go.Scatter3d(x=x_node, y=y_node, z=z_node, hovertext=labels, mode='markers', marker=dict(size=sizes, color=colors, opacity=1.0))

    amf_edge_scatter = go.Scatter3d(x=x_edge, y=y_edge, z=z_edge, mode='lines', line=dict(width=0.5, color='grey'))

    if nos_winners > 0:
        mean_distance = sum_distance / nos_winners
        mean_mapped = sum_mapped / nos_winners
    else:
        mean_distance = 0.0
        mean_mapped = 0

    fig = go.Figure(data=[neuron_scatter, amf_edge_scatter])
    fig.update_layout(width=1200, height=1200, title=dict(text=title),
                      scene=dict(xaxis_title='X Coord', yaxis_title='Y Coord', zaxis_title='Sequence'))
    print('Nos Mini Columns', len(fabric['neuro_columns']), 'Now Winners:', nos_winners, 'Mean Distance:', mean_distance, 'Mean n_BMU:', mean_mapped)
    fig.show()

### Instantiate the Fabric

In [ ]:
# the length of sequence to hold in short term memory
#
short_term_memory = 1

# lookback window threshold for detecting anomalies and motifs
#
mp_threshold = 0.10

# connection topology of neur_column in amfabric
#
structure = 'star'

# edges with probabilities below this threshold will be assumed zero and deleted
#
prune_threshold = 0.001

# a random seed used when growing the fabric and initialising neuro columns
#
random_seed = 221166

# instantiate an area of the amfabric for the selected client
#
amf = AMFabric(uid='client_x_trades',
               short_term_memory=short_term_memory,
               mp_threshold=mp_threshold,
               structure=structure,
               prune_threshold=prune_threshold,
               random_seed=random_seed)

# a list to collect the path of reasoning data
#
por_results = []

### Train with first SDR

In [ ]:
sdr_1 = SDR()

sdr_1.set_item(source_node=('trade', '*'),
               edge=('has_rgb', 'r'),
               target_node=('rgb', 'r'),
               probability=1.0,
               numeric=125,
               numeric_min=0,
               numeric_max=255
               )

sdr_1.set_item(source_node=('trade', '*'),
               edge=('has_rgb', 'g'),
               target_node=('rgb', 'g'),
               probability=1.0,
               numeric=125,
               numeric_min=0,
               numeric_max=255
               )

sdr_1.set_item(source_node=('trade', '*'),
               edge=('has_rgb', 'b'),
               target_node=('rgb', 'b'),
               probability=1.0,
               numeric=125,
               numeric_min=0,
               numeric_max=255
               )

    
por = amf.train(sdr=sdr_1, ref_id=1, fast_search=True, similarity_learn=True)
por_results.append(por)

# extract the current fabric
#
fabric = amf.decode_fabric()

# print specific level
#
plot_fabric_3d(fabric, 'client_x_trades', coords_to_highlight=None, neurons_to_plot=[0], short_term_memory=short_term_memory)


### Path of Reasoning data is also created

In [ ]:
por_results[-1]

### Extract a "Persist Graph"

In [ ]:
persist_graph = amf.get_persist_graph(ref_id=1, pg_to_update=None, only_updated=True)
print(persist_graph)

In [ ]:
persist_graph.plot(dimension=2, width=1700, height=1000)

In [ ]:
import os
from dotenv import load_dotenv
from src.kv_cache import KVGraphCache

load_dotenv()

config = {'db_name': os.getenv("DB_NAME"),
          'db_username': os.getenv("DB_USERNAME"),
          'db_password': os.getenv("DB_PASSWORD"),
          'db_system': os.getenv("DB_SYSTEM"),
          'db_config_file_path': os.getenv("DB_CONFIG_PATH"),
          'db_queries_file_path': os.getenv("DB_QUERIES_PATH"),
          }

in_memory_cache = KVGraphCache(config=config)

# save to cache
#
in_memory_cache.set_kv(store_name='amfabric', key='client_x_trades', value=persist_graph)

# ask cache to persist
#
in_memory_cache.persist()

### Train next SDR

In [ ]:
sdr_2 = SDR()
sdr_2.set_item(source_node=('trade', '*'),
               edge=('has_rgb', 'r'),
               target_node=('rgb', 'r'),
               probability=1.0,
               numeric=200,
               numeric_min=0,
               numeric_max=255
               )

sdr_2.set_item(source_node=('trade', '*'),
               edge=('has_rgb', 'g'),
               target_node=('rgb', 'g'),
               probability=1.0,
               numeric=200,
               numeric_min=0,
               numeric_max=255
               )

sdr_2.set_item(source_node=('trade', '*'),
               edge=('has_rgb', 'b'),
               target_node=('rgb', 'b'),
               probability=1.0,
               numeric=200,
               numeric_min=0,
               numeric_max=255
               )
    
por = amf.train(sdr=sdr_2, ref_id=2, fast_search=True, similarity_learn=True)
por_results.append(por)

# extract the current fabric
#
fabric = amf.decode_fabric()

# print specific level
#
plot_fabric_3d(fabric, 'client_x_trades', coords_to_highlight=None, neurons_to_plot=[0], short_term_memory=short_term_memory)


### Update existing persist graph and save to cache

In [ ]:
persist_graph = amf.get_persist_graph(ref_id=2, pg_to_update=persist_graph, only_updated=True)
print(persist_graph)

In [ ]:
in_memory_cache.set_kv(store_name='amfabric', key='client_x_trades', value=persist_graph)
in_memory_cache.persist()